In [10]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.stats import truncnorm
from numba import jit, njit
from shapely import MultiPoint, Point
from shapely.ops import nearest_points
import threading

Read dwelling types

In [9]:
dwellingTypes = pd.read_csv('./Data/DwellingTypesByZone2006.csv')

Read Zonal Profile data

In [10]:
profileData = pd.read_csv('./Data/GTAProfileData.csv')

Create ZoneSystem file which contains the zones to be used in the sim

In [11]:
zoneSystem = pd.DataFrame(data=profileData['DAUID'], columns=['DAUID'], dtype=str)
zoneSystem = zoneSystem[zoneSystem['DAUID'].str.len()==8]

Read Shapefile Zones

In [12]:
zoneShape:gpd.GeoDataFrame = gpd.read_file('./Data/TorontoZoneShape/TorontoZones_26917.shp')
zoneShape = zoneShape[zoneShape['DAUID'].astype(str).isin(zoneSystem['DAUID'])]

Create Zone Dataframe that holds all data

zoneData = pd.DataFrame(columns=['DAUID', 'Population', 'Employment rate', 'Area', "Total - Structural type of dwelling"])

for i, zone in zoneSystem.iterrows():
    numDwellings=200
    population = 400
    try:numDwellings = dwellingTypes.loc[dwellingTypes['Geography'].astype(str)==zone['DAUID'], 'Total - Structural type of dwelling'].iloc[0]
    except:pass
    try:population = int(profileData.loc[profileData['DAUID'].astype(str)==zone['DAUID'], "Population"].iloc[0])
    except:pass

    zoneData.loc[len(zoneData)] = [zone['DAUID'], population, profileData.loc[profileData['DAUID'].astype(str)==zone['DAUID'], "Employment rate"].iloc[0], float(zoneShape.loc[zoneShape['DAUID'].astype(str)==zone['DAUID'], 'geometry'].iloc[0].area), numDwellings]

zoneData.to_csv('./Data/combinedZoneData.csv', index=False)

<span style="font-size:5em;">CREATE PERSONS</span>

In [2]:
zoneData = pd.read_csv('./Data/combinedZoneData.csv')

Income generator

In [3]:
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)


In [4]:
salary_distribution_employed = get_truncated_normal(mean=70000, sd=40000, low=30000, upp=200000)
salary_distribution_retired = get_truncated_normal(mean=40000, sd=20000, low=15000, upp=70000)

In [5]:
totalPersons = zoneData['Population'].astype(float).sum()

In [6]:
employment_incomes = salary_distribution_employed.rvs(int(totalPersons*.8))
retired_incomes = salary_distribution_retired.rvs(int(totalPersons*.4))

<h3>synthesize persons<h3/>

initial person counts per zone

In [7]:
personZoneCounts = np.empty([0,2], dtype=np.float64)
for i, zone in zoneData.iterrows():
    personZoneCounts = np.r_[personZoneCounts, [[np.float64(zone['DAUID']), np.float64(zone['Population'])]]]

In [8]:
personZoneCounts.shape[0]

5526

Create numba function to synthesize persons

In [13]:
@jit(nopython=True)
def createPersons(args):
    num=args[0]
    personCounts = args[1]
    p = np.empty((0,10))
    for zoneNum in range(personCounts.shape[0]):
        if zoneNum%10==0:
            print("Thread ", num, ':', zoneNum)
        numPeopleInZone = personCounts[zoneNum][1]
        for person in range(numPeopleInZone):
            age = np.random.randint(0,100)
            gender = np.random.randint(0,2)
            # 0:SINGLE 1:MARRIED 2:CHILD
            relationship = 2 if age < 18 else np.random.randint(0, 2)
            occupation = 0 if age < 4 \
                    else 4 if age > 65 \
                    else 3 if age < 22 \
                    else 1 if np.random.random() < .70 \
                    else 2
            occupation_type = 1
            workplace = -1
            income=-1
            if occupation==1:
                income = employment_incomes[np.random.randint(0,int(totalPersons*.8))]
            elif occupation==4:
                income = retired_incomes[np.random.randint(0,int(totalPersons*.4))]
            
            schoolplace=-1
            
            p = np.append(p, np.array([[int(personZoneCounts[zoneNum][0]), -1, age, gender, relationship, occupation, occupation_type, workplace, income, schoolplace]]), axis=0)
    return p
    

Create multiple threads so it goes faster

In [11]:
class ReturnableThread(threading.Thread):
    # This class is a subclass of Thread that allows the thread to return a value.
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        self.args = args
        threading.Thread.__init__(self, group, target, name, args, kwargs)

        self.target = target
        self.result = None
    
    def run(self) -> None:
        self.result = self.target(self.args)


In [14]:
thread1 = ReturnableThread(target=createPersons, args=(1, personZoneCounts[:1500]))
thread2 = ReturnableThread(target=createPersons, args=(2, personZoneCounts[1500:3000]))
thread3 = ReturnableThread(target=createPersons, args=(3, personZoneCounts[3000:4500]))
thread4 = ReturnableThread(target=createPersons, args=(4, personZoneCounts[4500:6000]))

thread1.start()
thread2.start()
thread3.start()
thread4.start()

thread1.join()
thread2.join()
thread3.join()
thread4.join()

allPersons = np.concat([thread1.result, thread2.result, thread3.result, thread4.result], axis=0)

Thread  1 : 0
Thread  1 : 10
Thread  2 : 0
Thread  3 : 0
Thread  4 : 0
Thread  2 : 10
Thread  1 : 20
Thread  3 : 10
Thread  1 : 30
Thread  2 : 20
Thread  1 : 40
Thread  4 : 10
Thread  1 : 50
Thread  4 : 20
Thread  1 : 60
Thread  2 : 30
Thread  4 : 30
Thread  3 : 20
Thread  4 : 40
Thread  2 : 40
Thread  3 : 30
Thread  4 : 50
Thread  3 : 40
Thread  2 : 50
Thread  3 : 50
Thread  2 : 60
Thread  1 : 70
Thread  3 : 60
Thread  1 : 80
Thread  3 : 70
Thread  2 : 70
Thread  4 : 60
Thread  1 : 90
Thread  3 : 80
Thread  2 : 80
Thread  2 : 90
Thread  4 : 70
Thread  1 : 100
Thread  2 : 100
Thread  1 : 110
Thread  2 : 110
Thread  1 : 120
Thread  3 : 90
Thread  1 : 130
Thread  4 : 80
Thread  2 : 120
Thread  3 : 100
Thread  1 : 140
Thread  3 : 110
Thread  1 : 150
Thread  3 : 120
Thread  4 : 90
Thread  1 : 160
Thread  3 : 130
Thread  1 : 170
Thread  2 : 130
Thread  1 : 180
Thread  4 : 100
Thread  3 : 140
Thread  4 : 110
Thread Thread  1 : 190
Thread  2 : 140
Thread  1 : 200
Thread  2 : 150
Thread  4 : 1

Create dataframe of persons using the c-compiled function above to create the data

In [15]:
persons = pd.DataFrame(data=allPersons, columns=['id', 'hhId', 'age', 'gender', 'relationship', 'occupation', 'occupation_type', 'workplace', 'income', 'schoolplace'])

In [16]:
persons.to_csv('./Output/pp_2006.csv', index=False)

NATIONALITIES

In [11]:
persons = pd.read_csv('./Output/pp_2006.csv')

In [12]:
import re
import random
# taken from wikipedia general nationality distribution
tmp = """Canadian	746,960
English	732,555
Chinese	700,705
East Indian	643,370
Irish	544,380
Scottish	543,760
Italian	484,360
Filipino	274,675
German	271,815
French	247,790
Polish	237,240
Portuguese	210,420
Jamaican	200,335
Ukrainian	144,335
Russian	139,910
Pakistani	122,950
Spanish	106,685
Greek	99,145"""

countries = tmp.replace('\t', ' ').replace(',', '').split('\n')
nationalities = {}
total_nationalities_pop = 0
for i in countries:
    ind = re.search('[0-9]', i).span()[0]
    nationalities[i[:ind-1]] = 0.0
    total_nationalities_pop += int(i[ind:])

for i in countries:
    ind = re.search('[0-9]', i).span()[0]
    nationalities[i[:ind-1]] = int(i[ind:])/total_nationalities_pop

print(nationalities)
nationalities_list = random.choices(list(nationalities.keys()), weights=list(nationalities.values()),k=persons.shape[0])

{'Canadian': 0.11578280029575022, 'English': 0.11354994815070861, 'Chinese': 0.1086130275801029, 'East Indian': 0.09972579552623544, 'Irish': 0.08438181539172178, 'Scottish': 0.08428571207135206, 'Italian': 0.07507839395851126, 'Filipino': 0.042576096004116944, 'German': 0.04213278068757276, 'French': 0.03840877702324615, 'Polish': 0.036773470523406585, 'Portuguese': 0.032616226890639075, 'Jamaican': 0.03105299788107679, 'Ukrainian': 0.022372697976715095, 'Russian': 0.021686799278915085, 'Pakistani': 0.019057908450736974, 'Spanish': 0.016536746344586205, 'Greek': 0.015368005964606078}


In [13]:
persons['nationality'] = nationalities_list

In [18]:
persons['id'] = list(persons.index)

In [21]:
persons.head()

,id,hhId,age,gender,relationship,occupation,occupation_type,workplace,income,schoolplace,nationality
0,0,-1.0,73.0,0.0,1.0,4.0,1.0,-1.0,44260.521635,-1.0,Filipino
1,1,-1.0,5.0,1.0,2.0,3.0,1.0,-1.0,-1.000000,-1.0,Irish
2,2,-1.0,4.0,0.0,2.0,3.0,1.0,-1.0,-1.000000,-1.0,East Indian
3,3,-1.0,45.0,0.0,1.0,1.0,1.0,-1.0,105318.211510,-1.0,Italian
4,4,-1.0,58.0,0.0,0.0,1.0,1.0,-1.0,49226.216378,-1.0,Irish


In [22]:
persons.to_csv('./Output/pp_2006.csv', index=False)

In [ ]:
# del employment_incomes
# del retired_incomes
# del personZoneCounts
# del profileData
# del dwellingTypes

In [5]:
@njit
def test(num, start, end):
    print(num,start,end)